In [1]:
import pandas as pd
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
cfns = pd.read_csv('../data/classifications_cset.csv')

In [3]:
cfns.sample(5)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
21,2,6. Complete and final,2,False,A self-driving shuttle in Las Vegas was involv...,A self-driving public shuttle developed by Keo...,2017-11-08T08:00:00.000Z,2017-11-08T08:00:00.000Z,"Las Vegas, NV",Harm caused,...,Transportation and storage,False,Amateur,High,"[""Perception"",""Cognition"",""Action""]",environment sensors; self-driving automobile; ...,"[""Self-driving automobiles"",""Interpreting traf...","[""Vehicle/mobile robot""]","[""Robustness""]",True
33,2,6. Complete and final,2,False,An employee was laid off and blocked from acce...,"An employee was laid off, allegedly by an arti...",2018-07-01T07:00:00.000Z,2018-07-01T07:00:00.000Z,"Los Angeles, CA",Harm caused,...,NaN,False,NaN,NaN,[],NaN,"[""""]",[],[],True
48,5,6. Complete and final,2,False,In 2016 programmers created The Decentralized ...,"On June 18, 2016, an attacker successfully exp...",2016-06-18T07:00:00.000Z,2016-06-18T07:00:00.000Z,Global,Harm caused,...,Financial and insurance activities,False,Amateur,Low,"[""Cognition""]",Unclear,"[""Unclear""]","[""Software only""]","[""Specification""]",True
70,5,4. Peer review complete,6,False,"On February 14, 2016, a Google autonomous test...","On February 14, 2016, a Google autonomous test...",2016-02-14T08:00:00.000Z,2016-02-14T08:00:00.000Z,"Mountain View, CA",Harm caused,...,Transportation and storage,False,Amateur,High,"[""Perception"",""Action""]",deep learning; image classification; object de...,"[""computer vision"",""autonomous driving"",""self-...","[""Vehicle/mobile robot""]","[""Robustness""]",False
32,2,6. Complete and final,6,False,There are multiple reports of Amazon Alexa pro...,There are multiple reports of Amazon Alexa pro...,2018-01-01T00:00:00.000Z,2018-01-01T00:00:00.000Z,NaN,Harm caused,...,Information and communication,False,Amateur,Medium,"[""Perception"",""Cognition"",""Action""]",Amazon Alexa; natural language processing; vir...,"[""voice recognition"",""natural language process...","[""Consumer device""]","[""Robustness"",""Assurance""]",True


In [4]:
# clean up!
cfns.columns = cfns.columns.str.lower().str.replace('classifications.', '').str.replace(' ', '_')
cfns['beginning_date'] = pd.to_datetime(cfns['beginning_date'], utc=True).dt.date
cfns['ending_date'] = pd.to_datetime(cfns['ending_date'], utc=True).dt.date

def parse_list_columns(column):
    if type(column[0]) == str and column[0].startswith('['):
        return column.apply(eval)
    else: return column

cfns = cfns.apply(parse_list_columns)

In [5]:
cfns.columns

Index(['annotator', 'annotation_status', 'reviewer', 'quality_control',
       'full_description', 'short_description', 'beginning_date',
       'ending_date', 'location', 'near_miss', 'named_entities',
       'technology_purveyor', 'intent', 'severity', 'lives_lost',
       'harm_distribution_basis', 'harm_type', 'infrastructure_sectors',
       'financial_cost', 'laws_implicated', 'ai_system_description',
       'data_inputs', 'system_developer', 'sector_of_deployment',
       'public_sector_deployment', 'nature_of_end_user', 'level_of_autonomy',
       'relevant_ai_functions', 'ai_techniques', 'ai_applications',
       'physical_system', 'problem_nature', 'publish'],
      dtype='object')

In [6]:
cfns.shape

(96, 33)

In [7]:
cfns[cfns['publish'] == True].shape

(63, 33)

In [8]:
# harms: multiple options
cfns['harm_type'].apply(pd.Series).stack().value_counts()

Harm to social or political systems                           18
Harm to physical health/safety                                17
Harm to civil liberties                                       17
Psychological harm                                            17
Financial harm                                                 9
Harm to physical property                                      8
Harm to intangible property                                    3
Other:Privacy                                                  2
Other:Unable to detect facial features                         1
Other:Reputational harm                                        1
Other:Reputational harm; False incarceration                   1
Other:Harm to publicly available information                   1
Other:Reputational harm/social harm (libel and defamation)     1
dtype: int64

In [9]:
cfns['technology_purveyor'].apply(pd.Series).stack().value_counts()

Google                              18
Amazon                               6
Tesla                                4
Facebook                             4
Microsoft                            4
                                    ..
DataWorks Plus                       1
New York Department of Education     1
Niantic Labs                         1
Chicago Police Department            1
Outlook                              1
Length: 83, dtype: int64

In [15]:
cfns.columns

Index(['annotator', 'annotation_status', 'reviewer', 'quality_control',
       'full_description', 'short_description', 'beginning_date',
       'ending_date', 'location', 'near_miss', 'named_entities',
       'technology_purveyor', 'intent', 'severity', 'lives_lost',
       'harm_distribution_basis', 'harm_type', 'infrastructure_sectors',
       'financial_cost', 'laws_implicated', 'ai_system_description',
       'data_inputs', 'system_developer', 'sector_of_deployment',
       'public_sector_deployment', 'nature_of_end_user', 'level_of_autonomy',
       'relevant_ai_functions', 'ai_techniques', 'ai_applications',
       'physical_system', 'problem_nature', 'publish'],
      dtype='object')

In [16]:
for i in cfns.columns:
    u = cfns[i].unique()
    if len(u) < 10:
        print(i, u)
    else: print(i)

annotator [1 2 3 4 6 5]
annotation_status ['6. Complete and final' '3. In peer review' '4. Peer review complete']
reviewer [5 6 2 3]
quality_control [False  True]
full_description
short_description
beginning_date
ending_date
location
near_miss ['Harm caused' 'Unclear/unknown' 'Near miss']


TypeError: unhashable type: 'list'

In [22]:
cfns.head(1)

,classifications.Annotator,classifications.Annotation Status,classifications.Reviewer,classifications.Quality Control,classifications.Full Description,classifications.Short Description,classifications.Beginning Date,classifications.Ending Date,classifications.Location,classifications.Near Miss,...,classifications.Sector of Deployment,classifications.Public Sector Deployment,classifications.Nature of End User,classifications.Level of Autonomy,classifications.Relevant AI functions,classifications.AI Techniques,classifications.AI Applications,classifications.Physical System,classifications.Problem Nature,classifications.Publish
0,1,6. Complete and final,5,False,"On December 5, 2018, a robot punctured a can o...",Twenty-four Amazon workers in New Jersey were ...,2018-12-05T08:00:00.000Z,2018-12-05T08:00:00.000Z,"Robbinsville, NJ",Harm caused,...,Transportation and storage,False,NaN,Unclear/unknown,"[""Unclear""]",NaN,"[""robotics""]","[""Unknown/unclear""]","[""Unknown/unclear""]",True


In [42]:
s = cfns['classifications.Harm Type'].value_counts()
q = np.percentile(s, 50)
s[s > q].index

Index(['[]', '["Harm to social or political systems"]',
       '["Harm to physical health/safety"]', '["Psychological harm"]',
       '["Harm to civil liberties"]', '["Financial harm"]',
       '["Psychological harm","Harm to social or political systems"]',
       '["Harm to physical health/safety","Harm to physical property"]'],
      dtype='object')

In [44]:
s = cfns['classifications.Harm Type'].apply(eval).apply(pd.Series).stack().value_counts()
q = np.percentile(s, 50)
s[s > q].index

Index(['Harm to social or political systems', 'Harm to physical health/safety',
       'Harm to civil liberties', 'Psychological harm', 'Financial harm',
       'Harm to physical property'],
      dtype='object')

In [45]:
s

Harm to social or political systems                           18
Harm to physical health/safety                                17
Harm to civil liberties                                       17
Psychological harm                                            17
Financial harm                                                 9
Harm to physical property                                      8
Harm to intangible property                                    3
Other:Privacy                                                  2
Other:Reputational harm/social harm (libel and defamation)     1
Other:Reputational harm                                        1
Other:Unable to detect facial features                         1
Other:Reputational harm; False incarceration                   1
Other:Harm to publicly available information                   1
dtype: int64

In [17]:
cfns.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96 entries, 0 to 95
Data columns (total 33 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   annotator                 96 non-null     int64 
 1   annotation_status         96 non-null     object
 2   reviewer                  96 non-null     int64 
 3   quality_control           96 non-null     bool  
 4   full_description          96 non-null     object
 5   short_description         96 non-null     object
 6   beginning_date            95 non-null     object
 7   ending_date               94 non-null     object
 8   location                  79 non-null     object
 9   near_miss                 96 non-null     object
 10  named_entities            96 non-null     object
 11  technology_purveyor       96 non-null     object
 12  intent                    96 non-null     object
 13  severity                  96 non-null     object
 14  lives_lost                96